In [2]:
from selenium import webdriver
DRIVER_PATH = 'E:/Github/Data-Science-Projects/tensorflow keras/dog breeds/chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
import time
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [4]:
import requests, hashlib, io
from PIL import Image
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [5]:
import os
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
search_term = ['German Shepherd', 'Labrador Retriever', 'Golden Retriever', 'French Bulldog', 'Siberian Husky', 'Dalmatian', 'Shiba Inu', 'Samoyed', 'Cane Corso', 'Goldendoodle']
number_images = 100
for term in search_term:
    search_and_download(search_term=term, driver_path=DRIVER_PATH, number_images=number_images, target_path='./images')

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTUFjEHtwZJK9Umu2QOnr1avZ7NGrnkvxL0dw&usqp=CAU - as ./images\german_shepherd\b6d67569a0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRptTl-vhG_565YEO2RUX8Ocks8sUEyi-hTXw&usqp=CAU - as ./images\german_shepherd\1ec3c3df53.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJXz-OR7xMxvuSleSgu3HuZ8jWxVVZbQM7RA&usqp=CAU - as ./images\german_shepherd\63b8011ef9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSzd6czW1_91vwrXfHyjK5ag-aluyB6uPcLhg&usqp=CAU - as ./images\german_shepherd\09ca1c331c.jpg
SUCCESS - saved https://www.ctvnews.ca/polopoly_fs/1.5006198.1593547713!/httpImage/image.jpeg_gen/derivatives/landscape_960/image.jpeg - as ./images\german_shepherd\1786d80c40.jpg
SUCCESS - saved https://germanshepherdsowner.com/wp-content/uploads/2019/10/127516466_l.jpg

SUCCESS - saved https://smartcdn.prod.postmedia.digital/vancouversun/wp-content/uploads/2020/02/dog_show-w.jpg - as ./images\german_shepherd\f43f817f09.jpg
SUCCESS - saved https://3xnhi43vikn244hoyg160zl4-wpengine.netdna-ssl.com/wp-content/uploads/2020/04/21232700_web1_200415-AVN-SPCA-abandoned-puppies-alberni_1-768x1024.jpg - as ./images\german_shepherd\75d2ce801e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ29qXoN7tqFPJx0A7CAFs77kDr-x_3kzo1WQ&usqp=CAU - as ./images\german_shepherd\6bbe7255cc.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQk16YBXZGIu9CES8x2RxdVJ-KAilEFkchUww&usqp=CAU - as ./images\german_shepherd\e428690530.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/German-shepherd-4040871920.jpg/450px-German-shepherd-4040871920.jpg - as ./images\german_shepherd\17cc443424.jpg
SUCCESS - saved https://www.yourpurebredpuppy.com/dogbreeds/photos2-G/german-shepherd-07.jpg - as ./images\german_sheph

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQZXOfqv6xkzi01FOauPcmNQ03avXxOqJp5Tw&usqp=CAU - as ./images\german_shepherd\775a413021.jpg
Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR2PBP4kPMrlu4C3zet04m-AsouGyoer3uH9A&usqp=CAU - as ./images\labrador_retriever\5fda6d9ca7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSkuS-7dXb3JFe6kIAfl8EWMI7Mxo7uzniO5A&usqp=CAU - as ./images\labrador_retriever\82d473df02.jpg
SUCCESS - saved https://epi.azureedge.net/website-images/images/default-album/labrador-retrieverc114b6a24c9f653bbdf5ff0000035de2.jpg?sfvrsn=98bd37b_2 - as ./images\labrador_retriever\a7873fba97.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcStBoRtkudp6pkGLdnww0bl6k4Lo_ZBfJmIQA&usqp=CAU - as ./images\labrador_retriever\23d72b0ba6.jpg
SUCCESS - saved https://i.ytimg.com/vi/RKD3fakO6hs/maxresdefault

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTENDKiPI9JfmDqnPVnVu7orUWpdXoRV4uaCg&usqp=CAU - as ./images\labrador_retriever\ea5a2ddda2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWnFbuRkB_FXyL90_8xnaFy_AYQhdMbbMnQQ&usqp=CAU - as ./images\labrador_retriever\bb74d7d68b.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVW90oIuZVZcjJWxOB_IVuwFfqh9zFNqZPqA&usqp=CAU - as ./images\labrador_retriever\17d4f8a645.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSS0nhwOpmYfzL7stTtrcqxNibJbwF2wodkQw&usqp=CAU - as ./images\labrador_retriever\c08c05d558.jpg
SUCCESS - saved https://goldenhearts.co/wp-content/uploads/2019/05/golden-retriever-vs-labrador-1.jpg - as ./images\labrador_retriever\87f483217c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTU6lPIf1jLntMbZfxVoV84OmQhVeaYZ0j5lg&usqp=CAU - as ./images\labrador_retriever\984b38672e.jpg
SUCCESS - saved https://encryp

c:\users\jiahu\documents\plaidml\venv\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

SUCCESS - saved https://grcc.net/wp-content/uploads/2015/06/Dogs-for-Rob-1-e1434850228704.jpg - as ./images\golden_retriever\098faa8a12.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQQTxfB6JXrVfNIg9SybZJHpTd8abwGG4F3ug&usqp=CAU - as ./images\golden_retriever\441b03bdb4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9OxaUasu3UWEtZS9v3iZRP8BrqMknc_4khg&usqp=CAU - as ./images\golden_retriever\8523bdf5e8.jpg
SUCCESS - saved https://i.ytimg.com/vi/PBZtSLsFo-s/maxresdefault.jpg - as ./images\golden_retriever\7e12476c18.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTZ-4kUMvlC78qGewA6aeh0rlqvZp2KAeA0VQ&usqp=CAU - as ./images\golden_retriever\1740f1b5e3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSYUS2lkFOb6-7tdoUwE0-vw6CrRBR4syQz8w&usqp=CAU - as ./images\golden_retriever\2ecb7b1d22.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSMBqvysGblsm4vl_aShnKxsPWH0

SUCCESS - saved https://i.pinimg.com/originals/0c/e8/cd/0ce8cddcbbb19d11199fa8ae2d81927f.jpg - as ./images\golden_retriever\2ddd0a0f74.jpg
SUCCESS - saved https://thumbs-prod.si-cdn.com/12ZiEwalyjAleLtrh3Dk2IEWoj8=/1072x720/filters:no_upscale()/https://public-media.si-cdn.com/filer/2d/f0/2df0c602-21b2-4c98-829e-2fe7bd98f25d/augie_celebrating_20_years_old.jpg - as ./images\golden_retriever\fa5f0bd537.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQoI_7mFRBYBWAEDJB1XWAtJ6ZnkMFzFXb-jQ&usqp=CAU - as ./images\golden_retriever\6281061180.jpg
SUCCESS - saved https://cdn.rona.ca/webassets/images/330685715_MainImage_001_l.jpg - as ./images\golden_retriever\0429995c76.jpg
SUCCESS - saved https://firsttimedogmom.com/wp-content/uploads/2019/04/what-is-bad-about-golden-retrievers-e1554233820292.jpg - as ./images\golden_retriever\d53de244bd.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW8CvIxH9L-02GzqUW3BCJX6w6eE_eNpCxXQ&usqp=CAU - as ./images\

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ6BlCMoaBWO3QYfrQ_zYjAZPwMAhDgjWugxw&usqp=CAU - as ./images\french_bulldog\b653df8015.jpg
SUCCESS - saved https://i.guim.co.uk/img/media/a90bff2e310a0b2934b6793c90966333172e1038/0_21_2424_1454/master/2424.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=89fb2a3a48e00a9c055e6efa6752cff1 - as ./images\french_bulldog\f9cf45945d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTy_XppqlHUOVkD6bLoseS3Sh2yV5_WRGkhYA&usqp=CAU - as ./images\french_bulldog\9a79c5d3b8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ6zep0QAesbqiJzbHhEmH7z6vDQ67kHV9wIw&usqp=CAU - as ./images\french_bulldog\493cc0d3a2.jpg
SUCCESS - saved https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2020/09/30171012/French-Bulldog-puppies-being-held-in-the-arms-of-a-woman.jpeg - as ./images\french_bulldog\43dc468824.jpg
SUCCESS - saved https://i.etsystatic.com/13844389/r/il/d3fb9a

SUCCESS - saved https://dynamic.indigoimages.ca/gifts/4055744020599_hi.jpg?scaleup=true&width=614&maxheight=614 - as ./images\french_bulldog\7223d20d46.jpg
SUCCESS - saved https://eurekalert.org/multimedia_ml/pub/web/5322_web.jpg - as ./images\french_bulldog\422c2c0df7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSKuCE88K4INYIwYL58ZyZvqXdkk6Mw2osgrw&usqp=CAU - as ./images\french_bulldog\772c5a3f17.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSMKbjP-wOyHb7L0NakcHvUXKJ8lXBsKWNcHA&usqp=CAU - as ./images\french_bulldog\0217bc4dc5.jpg
SUCCESS - saved https://nypost.com/wp-content/uploads/sites/2/2020/05/bulldog.jpg?quality=80&strip=all - as ./images\french_bulldog\9c08571d1b.jpg
SUCCESS - saved https://i.pinimg.com/originals/92/db/a1/92dba1b6e3ca1dae7e69ad545f5f3d04.jpg - as ./images\french_bulldog\4cdaca0d2b.jpg
ERROR - Could not save https://www.talenthounds.ca/wp-content/uploads/formidable/SABA-This-One.jpg - cannot identify image

SUCCESS - saved https://s3.amazonaws.com/petcentral.com/wp-content/uploads/2016/09/04043631/Siberian-Husky2.jpg - as ./images\siberian_husky\a940081f67.jpg
SUCCESS - saved https://cf.ltkcdn.net/dogs/images/std/258863-425x283-brown-husky.jpg - as ./images\siberian_husky\a353d7c4a3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSey9ZBi_a1wCN95Uc2tHTNlrUX9SUtbv--ug&usqp=CAU - as ./images\siberian_husky\c89f04fe36.jpg
SUCCESS - saved https://t1.pixers.pics/img-c676e9e9/posters-siberian-husky-dog-puppies.jpg?H4sIAAAAAAAAA42PWU7FMAxFt9NKae0MzdAFvN-3hCpN0kehQ5QUqFg9KSD-kJA_bF_Z5-rC65btFMCF7QgJ1tn7JcA0L2XLfQp5_ggVEq5F3Rd1qRCx7ve3kFzaY9UIQRrKFZFUko7xun-35XO16aV6Oo6Ye4DM2zifBVeay-DWDAypAtTQGW2Msp4LZ8QQlyYfdvM2-UbiKbCN24PgVf_naqAI3ejpKEftUYx6YAzPDptfwheWa02EYhf6J6VCJOpKd6R5rUrcvVwf1XN81PCH5_cM5Qtud5AGlATBgV7KcLtLo6TgdJBaGc7QKsMcd84YV_JOTjsvAwrK2-LxCSS-AByJAQAA - as ./images\siberian_husky\bff6c31ebc.jpg
SUCCESS - saved https://i.etsystatic.com/5302623/r/il/38e94f/126840